In [2]:
import duckdb
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pandas as pd

DATA_OCT = "/home/angels/cleanbig/data/2019-Oct_optimized.parquet"
DATA_NOV = "/home/angels/cleanbig/data/2019-Nov_optimized.parquet"



In [3]:
dataset_oct = ds.dataset(DATA_OCT, format="parquet")
batch = next(dataset_oct.to_batches(batch_size=5000))
preview = batch.to_pandas()
preview.head()


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00+00:00,2,44600062,2103807459595387724,-1,2773,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00+00:00,2,3900821,2053013552326770905,29,167,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01+00:00,2,17200506,2053013559792632471,111,-1,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01+00:00,2,1307067,2053013558920217191,70,1778,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04+00:00,2,1004237,2053013555631882655,97,165,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [4]:
import pyarrow.parquet as pq

pf = pq.ParquetFile(DATA_OCT)
pf.schema


required group field_id=-1 schema {
  optional int64 field_id=-1 event_time (Timestamp(isAdjustedToUTC=true, timeUnit=nanoseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional int32 field_id=-1 event_type;
  optional int32 field_id=-1 product_id;
  optional int64 field_id=-1 category_id;
  optional int32 field_id=-1 category_code;
  optional int32 field_id=-1 brand;
  optional float field_id=-1 price;
  optional int32 field_id=-1 user_id;
  optional binary field_id=-1 user_session (String);
}

In [5]:
con = duckdb.connect()

row_count = con.execute(f"SELECT COUNT(*) FROM '{DATA_OCT}'").fetchone()[0]
row_count


42448764

In [6]:
con.execute(f"""
    SELECT event_type, COUNT(*) AS cnt
    FROM '{DATA_OCT}'
    GROUP BY event_type
    ORDER BY cnt DESC
""").df()


,event_type,cnt
0,2,40779399
1,0,926516
2,1,742849


In [7]:
con.execute(f"""
    SELECT 
        MIN(price) AS min_price,
        AVG(price) AS avg_price,
        MAX(price) AS max_price,
        APPROX_QUANTILE(price, 0.5) AS median_price
    FROM '{DATA_OCT}'
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,min_price,avg_price,max_price,median_price
0,0.0,290.323661,2574.070068,163.692244


In [8]:
con.execute(f"""
    SELECT category_code, COUNT(*) AS freq
    FROM '{DATA_OCT}'
    GROUP BY category_code
    ORDER BY freq DESC
    LIMIT 20
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,category_code,freq
0,-1,13515609
1,97,11507231
2,96,1311033
3,70,1137623
4,101,1113750
5,90,1100188
6,45,887755
7,48,869404
8,28,801670
9,11,763901


In [9]:
con.execute(f"""
    SELECT brand, COUNT(*) AS freq
    FROM '{DATA_OCT}'
    GROUP BY brand
    ORDER BY freq DESC
    LIMIT 20
""").df()


,brand,freq
0,-1,6117080
1,2701,5282775
2,165,4122554
3,3358,3083763
4,1395,1111205
5,1846,655861
6,1790,562404
7,457,557090
8,2312,482887
9,2860,456644


In [10]:
con.execute(f"""
    SELECT 
        DATE_TRUNC('day', event_time) AS day,
        COUNT(*) AS events
    FROM '{DATA_OCT}'
    GROUP BY day
    ORDER BY day
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,day,events
0,2019-10-01 00:00:00+02:00,1230402
1,2019-10-02 00:00:00+02:00,1192377
2,2019-10-03 00:00:00+02:00,1125472
3,2019-10-04 00:00:00+02:00,1415900
4,2019-10-05 00:00:00+02:00,1328554
5,2019-10-06 00:00:00+02:00,1321393
6,2019-10-07 00:00:00+02:00,1202095
7,2019-10-08 00:00:00+02:00,1369372
8,2019-10-09 00:00:00+02:00,1346685
9,2019-10-10 00:00:00+02:00,1282164


In [11]:
sample_df = con.execute(f"""
    SELECT *
    FROM '{DATA_OCT}'
    USING SAMPLE 0.01  -- ~1% sample
""").df()

sample_df.head()


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session


In [13]:
import pyarrow.parquet as pq
import pyarrow as pa


pf = pq.ParquetFile(DATA_OCT)
schema = pf.schema_arrow

category_maps = {}

for col in ["event_type", "category_code", "brand"]:
    field = schema.field(col)
    if pa.types.is_dictionary(field.type):
        dict_values = field.type.index_dictionary.to_pylist()
        
        # Inverse mapping: int -> original string
        category_maps[col] = {i: v for i, v in enumerate(dict_values)}

category_maps


{}

In [16]:
path = "./data/2019-Oct_optimized.parquet"

import duckdb
duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet('{path}') LIMIT 0")


IOException: IO Error: No files found that match the pattern "./data/2019-Oct_optimized.parquet"

In [18]:
import os

data_files = os.listdir("./data")
data_files



FileNotFoundError: [Errno 2] No such file or directory: './data'

In [19]:
os.listdir("/home/angels/cleanbig/data")


['2019-Nov_optimized.parquet',
 '2019-Oct_optimized.parquet',
 '2019-Nov.csv',
 '2019-Oct.csv']

In [20]:
os.getcwd()

'/home/angels/cleanbig/scripts'

In [23]:
os.listdir("../data")

['2019-Nov_optimized.parquet',
 '2019-Oct_optimized.parquet',
 '2019-Nov.csv',
 '2019-Oct.csv']

In [22]:
os.listdir("/home/angels/cleanbig/data")

['2019-Nov_optimized.parquet',
 '2019-Oct_optimized.parquet',
 '2019-Nov.csv',
 '2019-Oct.csv']

In [1]:
import pyarrow.parquet as pq
import pyarrow as pa

path = "../data/2019-Oct_optimized.parquet"

parquet_file = pq.ParquetFile(path)
schema = parquet_file.schema_arrow
schema


event_time: timestamp[ns, tz=UTC]
event_type: int32
product_id: int32
category_id: int64
category_code: int32
brand: int32
price: float
user_id: int32
user_session: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1168